<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
using Pkg; Pkg.activate(".")
Pkg.instantiate()

  Activating project at `a:\07Dissertation\01 Seminare\AMO\Abgabe\Team_assignment_AMO`

┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.10.0). Unexpected behavior may occur.
└ @ nothing A:\07Dissertation\01 Seminare\AMO\Abgabe\Team_assignment_AMO\Manifest.toml:0


   Installed CodecBzip2 ───────── v0.8.2
   Installed CodecZlib ────────── v0.7.4


   Installed TranscodingStreams ─ v0.10.3


   Installed JuMP ─────────────── v1.19.0


...

  ✓ CodecBzip2


  ✓ IJulia


  ✓ MathOptInterface


  ✓ HiGHS


  ✓ JuMP
  9 dependencies successfully precompiled in 140 seconds. 56 already precompiled.


In [9]:
using JuMP,HiGHS,DataFrames


In [77]:
#I=1:3                                  #number of generation units
#Q=1:1                                  #number of stochastic generation units
#T=1:1                                  #number of time intervalls 
#Ω=1:2                                  #number of scenarios
#J=1:2                                  #number of loads
#Λ=1:1                                  #number of buses
#C=[100, 80, 50]                         #unit cost of generation
#C_RU=[20, 10, 15]                       #cost of up reserve commitment
#C_RD=[25, 15, 15]                       #cost of down reserve commitment
#C_U=[10, 15, 20]                        #cost of up reserve commitment
#C_D=[30, 10, 15]                        #cost of down reserve commitment
#π=[0.7, 0.3]                           #probabilities of the scenarios
#V_LOL=[20, 25]                          #value of the lost load
#C_cur=[15]                              #cost of curtailment at turbine
#L= 90 .+ 30 * randn(2, 1)               #loads for every time intervall
#W_realized = 70 .+ 15 * randn(1, 2, 1)  #create random scenarios for four time points
#P_max=[70, 80, 150];                    #maximum production capacity

I=1:3                                  #number of generation units
Q=1:1                                  #number of stochastic generation units
T=1:1                                  #number of time intervalls 
Ω=1:2                                  #number of scenarios
J=1:2                                  #number of loads
Λ=1:2                                  #number of buses
C=[10, 30, 35]                          #unit cost of generation
C_RU=[16, 13, 10]                       #cost of up reserve commitment
C_RD=[15, 12, 9]                        #cost of down reserve commitment
C_U=[10, 30, 35]                        #cost of up reserve commitment
C_D=[10, 30, 35]                        #cost of down reserve commitment
π=[0.6, 0.4]                           #probabilities of the scenarios
V_LOL=[200, 200]                        #value of the lost load
C_cur=[0] #[15]                         #cost of curtailment at turbine
L= [40,100]               #loads for every time intervall
W_realized = [[[50] [10]] [[50] [10]] [[50] [10]]] #create random scenarios for four time points q,ω,t
#[[[111 112] [121 122]] [[211 212] [221 222]] [[311 312] [321 322]]]
P_max=[50, 110, 100];                   #maximum production capacity
b = [[1, 1/0.13] [1/0.13, 1]]                     #loss of transmission
LC_Max = [[0,100] [100, 0]]          #maximum transmission quantity between two buses

2×2 Matrix{Int64}:
   0  100
 100    0

In [78]:
data= Dict(
    :I => I,
    :Q => Q,
    :T => T,
    :Ω => Ω,
    :J => J,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :P_max => P_max,
    :C_U => C_U,
    :C_D => C_D,
    :π => π,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max,
    :b => b,
    :LC_Max => LC_Max
    );
print(W_realized[1,2,1])

10

In [85]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[i in data[:I], t in data[:T]] >=0)
    R_U=@variable(model, R_U[i in data[:I], t in data[:T]] >=0)
    R_D=@variable(model, R_D[i in data[:I], t in data[:T]] >=0)
    r_U=@variable(model, r_U[i in data[:I], ω in data[:Ω], t in data[:T]] >=0)
    r_D=@variable(model, r_D[i in data[:I], ω in data[:Ω], t in data[:T]] >=0)
    L_Shed=@variable(model, L_Shed[j in data[:J], ω in data[:Ω], t in data[:T]] >=0)
    W_spill=@variable(model, W_spill[q in data[:Q], ω in data[:Ω], t in data[:T]] >=0)
    W_s=@variable(model, W_s[q in data[:Q], t in data[:T]] >=0)
    #Delta=@variable(model, Delta[i in data[:Λ], j in data[:Λ]])
    #PF=@variable(model, PF[ω in data[:Ω], i in data[:Λ], j in data[:Λ]])
    
    vars= Dict(
        :P=> P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s
    )
    
    return vars
end;

"""
:Delta => Delta
:PF => PF
"""

":Delta => Delta\n:PF => PF\n"

In [86]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
            c1[i in data[:I], t in data[:T]], vars[:P][i, t]+ vars[:R_U][i, t]<= P_max[i]
            c2[i in data[:I] ,t in data[:T]], vars[:P][i, t]- vars[:R_D][i, t]>= 0
            c3[i in data[:I], t in data[:T], ω in data[:Ω]], vars[:r_U][i, ω, t] <= vars[:R_U][i,t]
            c4[i in data[:I], t in data[:T], ω in data[:Ω]], vars[:r_D][i, ω, t] <= vars[:R_D][i,t]
            Power_balance_day_ahead[t in data[:T]], 
            sum(vars[:P][i,t] for i in data[:I]) + sum(vars[:W_s][q, t] for q in data[:Q]) - sum(L[j, t] for j in data[:J]) == 0
            Power_balance_at_stage[ω in data[:Ω], t in data[:T]], 
            sum(vars[:r_U][i, ω, t] for i in data[:I]) - sum(vars[:r_D][i, ω, t] for i in data[:I]) - 
            sum(vars[:L_Shed][j, ω, t] for j in data[:J]) + sum(data[:W_realized][q,ω,t] -vars[:W_s][q, t] for q in data[:Q])== 0      
    end)
end;

"""
c5[ω in data[:Ω], b1 in data[:Λ], b2 in data[:Λ], vars[:PF[b1, b2]] <= data[:LC_Max[b1,b2]]]
c6[ω in data[:Ω], b1 in data[:Λ], b2 in data[:Λ], vars[:Delta[b1, b2]] - vars[:Delta[b2, b1]] * data[:b[b1, b2]] == vars[:PF[b1, b2]]]
- vars[:PF[b1, b2]]
"""

"c5[ω in data[:Ω], b1 in data[:Λ], b2 in data[:Λ], vars[:PF[b1, b2]] <= data[:LC_Max[b1,b2]]]\nc6[ω in data[:Ω], b1 in data[:Λ], b2 in data[:Λ], vars[:Delta[b1, b2]] - vars[:Delta[b2, b1]] * data[:b[b1, b2]] == vars[:PF[b1, b2]]]\n- vars[:PF[b1, b2]]\n"

In [87]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][i,t]* C[i] + vars[:R_U][i,t] * C_RU[i] + vars[:R_D][i,t] * C_RD[i] for i in data[:I], t in data[:T]) +
    sum(π[ω] * (sum(data[:C_U][i] * vars[:r_U][i, ω ,t] + data[:C_D][i] * vars[:r_D][i, ω, t] for i in data[:I], t in data[:T]) +
                sum(data[:C_cur][q] * vars[:W_spill][q, ω, t] for q in data[:Q], t in data[:T]) 
                + sum(data[:V_LOL][j] * vars[:L_Shed][j, ω, t] for j in data[:J], t in data[:T])) for ω in data[:Ω])
    )
end;

In [88]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

Min 10 P[1,1] + 16 R_U[1,1] + 15 R_D[1,1] + 30 P[2,1] + 13 R_U[2,1] + 12 R_D[2,1] + 35 P[3,1] + 10 R_U[3,1] + 9 R_D[3,1] + 6 r_U[1,1,1] + 6 r_D[1,1,1] + 18 r_U[2,1,1] + 18 r_D[2,1,1] + 21 r_U[3,1,1] + 21 r_D[3,1,1] + 120 L_Shed[1,1,1] + 120 L_Shed[2,1,1] + 4 r_U[1,2,1] + 4 r_D[1,2,1] + 12 r_U[2,2,1] + 12 r_D[2,2,1] + 14 r_U[3,2,1] + 14 r_D[3,2,1] + 80 L_Shed[1,2,1] + 80 L_Shed[2,2,1]
Subject to
 Power_balance_day_ahead[1] : P[1,1] + P[2,1] + P[3,1] + W_s[1,1] == 140
 Power_balance_at_stage[1,1] : r_U[1,1,1] + r_U[2,1,1] + r_U[3,1,1] - r_D[1,1,1] - r_D[2,1,1] - r_D[3,1,1] - L_Shed[1,1,1] - L_Shed[2,1,1] - W_s[1,1] == -50
 Power_balance_at_stage[2,1] : r_U[1,2,1] + r_U[2,2,1] + r_U[3,2,1] - r_D[1,2,1] - r_D[2,2,1] - r_D[3,2,1] - L_Shed[1,2,1] - L_Shed[2,2,1] - W_s[1,1] == -10
 c2[1,1] : P[1,1] - R_D[1,1] >= 0
 c2[2,1] : P[2,1] - R_D[2,1] >= 0
 c2[3,1] : P[3,1] - R_D[3,1] >= 0
 c1[1,1] : P[1,1] + R_U[1,1] <= 50
 c1[2,1] : P[2,1] + R_U[2,1] <= 110
 c1[3,1] : P[3,1] + R_U[3,1] <= 100
 c3[1,

In [89]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
21 rows, 24 cols, 56 nonzeros
20 rows, 23 cols, 56 nonzeros
Presolve : Reductions: rows 20(-1); columns 23(-5); elements 56(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -9.9999686469e+00 Ph1: 3(3); Du: 1(9.99997) 0s
          7     2.6600000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  2.6600000000e+03
HiGHS run time      :          0.00


In [97]:
# Print the values of the variables
println("Variable values:")
println("P:")
for i in data[:I], t in data[:T]
    println("P[$i]: ", value(vars[:P][i, t]))
end

println("\nR_U:")
for i in data[:I], t in data[:T]
    println("R_U[$i]: ", value(vars[:R_U][i, t]))
end

println("\nR_D:")
for i in data[:I], t in data[:T]
    println("R_D[$i]: ", value(vars[:R_D][i, t]))end

println("\nL_Shed:")
for j in data[:J], ω in data[:Ω], t in data[:T]
    println("L_Shed[$j, $ω, $t]: ", value(vars[:L_Shed][j, ω, t]))
end

println("\nW_spill:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_spill[$q, $ω, $t]: ", value(vars[:W_spill][q, ω, t]))
end

println("\nW_realized:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_realized[$q, $ω, $t]: ", value(data[:W_realized][q, ω, t]))
end

println("\nW_s:")
for q in data[:Q], t in data[:T]
    println("W_s[$q, $t]: ", value(vars[:W_s][q, t]))
end

println("\nr_U:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_U[$i, $ω, $t]: ", value(vars[:r_U][i, ω, t]))
end

println("\nr_D:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_D[$i, $ω, $t]: ", value(vars[:r_D][i, ω, t]))
end

println("\nLoad")
for j in data[:J]
    print("L[$j]:", data[:L][j])
end

print("\n\nsum:", sum(data[:L]))

"""
print("Day-ahead Energy Cost" & sum(value(vars[:P][i,t])* C[i] for i in data[:I], t in data[:T]))

print("Reserve capacity costs" & vars[:R_U][i,t] * C_RU[i] + vars[:R_D][i,t] * C_RD[i] for i in data[:I], t in data[:T])

print("Balancing costs in scenarios" & sum(π[ω] * (sum(data[:C_U][i] * vars[:r_U][i, ω ,t] + data[:C_D][i] * vars[:r_D][i, ω, t] for i in data[:I], t in data[:T]) +
            sum(data[:C_cur][q] * vars[:W_spill][q, ω, t] for q in data[:Q], t in data[:T]) 
            + sum(data[:V_LOL][j] * vars[:L_Shed][j, ω, t] for j in data[:J], t in data[:T])) for ω in data[:Ω]))
"""

Variable values:
P:
P[1]: 50.0


P[2]: 40.0
P[3]: 0.0

R_U:
R_U[1]: 0.0
R_U[2]: 0.0
R_U[3]: 40.0

R_D:


R_D[1]: 0.0
R_D[2]: 0.0
R_D[3]: 0.0

L_Shed:
L_Shed[1, 1, 1]: 0.0
L_Shed[1, 2, 1]: 0.0
L_Shed[2, 1, 1]: 0.0
L_Shed[2, 2, 1]: 0.0

W_spill:


W_spill[1, 1, 1]: 0.0
W_spill[1, 2, 1]: 0.0

W_realized:
W_realized[1, 1, 1]: 50
W_realized[1, 2, 1]: 10

W_s:


W_s[1, 1]: 50.0

r_U:
r_U[1, 1, 1]: 0.0
r_U[1, 2, 1]: -0.0
r_U[2, 1, 1]: 0.0
r_U[2, 2, 1]: -0.0


r_U[3, 1, 1]: 0.0
r_U[3, 2, 1]: 40.0

r_D:
r_D[1, 1, 1]: 0.0
r_D[1, 2, 1]: 0.0
r_D[2, 1, 1]: 0.0
r_D[2, 2, 1]: 0.0
r_D[3, 1, 1]: 0.0


r_D[3, 2, 1]: 0.0

Load
L[1]:40L[2]:100

sum:140

"print(\"Day-ahead Energy Cost\" & sum(value(vars[:P][i,t])* C[i] for i in data[:I], t in data[:T]))\n\nprint(\"Reserve capacity costs\" & vars[:R_U][i,t] * C_RU[i] + vars[:R_D][i,t] * C_RD[i] for i in data[:I], t in data[:T])\n\nprint(\"Balancing costs in scenarios\" & sum(π[ω] * " ⋯ 52 bytes ⋯ "D][i] * vars[:r_D][i, ω, t] for i in data[:I], t in data[:T]) +\n            sum(data[:C_cur][q] * vars[:W_spill][q, ω, t] for q in data[:Q], t in data[:T]) \n            + sum(data[:V_LOL][j] * vars[:L_Shed][j, ω, t] for j in data[:J], t in data[:T])) for ω in data[:Ω]))\n"